# AIB to Bambora SF Cost Comparison Application

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


## Import the Necessary Libraries
Below are the libraries required to assemble the project.

In [2]:
import pyodbc #CONNECT VIA PYODBC IMPORT THE PYODBC LIBRARY
import pandas as pd #CONNECT TO THE PANDAS LIBRARY
import numpy as np #IMPORT NUMPY
import csv
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:,.2f}'.format
print pyodbc.dataSources()
conn=C.connect("Driver={NetezzaSQL};SERVER=10.80.10.70;PORT=5480;DATABASE=P_FDW;UID=djohnson;PWD=Djohnson#1234",ansi=True)

{'ECL_CODA11': 'Oracle in instantclient_11_2', 'NZSQL': 'NetezzaSQL', 'IST_CODA11': 'Oracle in instantclient_11_2', 'BO_Lookup': 'Oracle in instantclient_11_2', 'Excel Files': 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'dBASE Files': 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'MS Access Database': 'Microsoft Access Driver (*.mdb, *.accdb)'}


## Bambora Pull

As the data is queried the transaction amounts are rounded.<br>
The transaction amounts are rounded to allow for flow data to be matched between AIB and Bambora.<br>
For example, England - Netherlands where the transaction amount is 20.00 and not 20.15 or 20.13 is much easier to match.<br>

The non-uniformity of transaction amounts that were not rounded lead to memory issues.<br>
The process was too laborious because the unqiue transaction amounts led to very large datasets have to be queried against another very large dataset.<br>
This decision allows for efficiency.

In [61]:
#Bambora
bb=pd.read_sql(
'''
SELECT 
bb.BAMBORA_REF
,TO_CHAR(bb.SETTLEMENT_DATE, 'MM') AS MONTH
,TO_CHAR(bb.SETTLEMENT_DATE, 'DD') AS DAY
,TO_CHAR(bb.SETTLEMENT_DATE, 'YYYY') AS YEAR
,TO_CHAR(bb.SETTLEMENT_DATE,'YYYY-MM') AS PERIOD
,bb.CONTRACT_ID, bb.ISSUER_COUNTRY_NAME, bb.MERCHANT_COUNTRY_NAME,bb.TRANSACTION_TYPE,bb.CARD_SCHEME,bb.CREDIT_DEBIT,bb.CONSUMER_CORPORATE
,bb.SF_FIXED_FEE_RESULT,bb.SF_PERC_FEE_RESULT,bb.TRANSACTION_TYPE AS B_TRANSACTION_TYPE,bb.CONSUMER_CORPORATE AS C_TYPE
,bb.VOLUME AS VOLUME,ROUND(bb.TRANSACTION_AMOUNT_EUR) AS FLOW,bb.SCHEME_TOTAL_FEE_EUR AS SF,bb.SF_FIXED_FEE,bb. SF_PERC_FEE,bb.TRANSACTION_AMOUNT_EUR
FROM DATA.ACQUIRER_BAMBORA_COST bb
WHERE bb.TRANSACTION_TYPE='TRANSACTION'
AND bb.SETTLEMENT_DATE BETWEEN '11-01-2017' AND '12-31-2017'
'''
,conn)

## AIB Pull
As the data is queried the transaction amounts are rounded.<br>
The transaction amounts are rounded to allow for flow data to be matched between AIB and Bambora.<br>

The AIB is Averaged and the Flow amount is rounded

In [62]:
#AIB
ab=pd.read_sql(
'''
SELECT 
MAX(TRANSACTION_DATE) AS TRANSACTION_DATE,
ab.CONTRACT_ID, ab.ISSUER_COUNTRY, ab.CARD_BRAND, ab.CLEARING_COUNTRY_NAME,ab.TRANSACTION_TYPE
,ROUND(ab.TRANSACTION_FLOW_EUR) AS FLOW,AVG((ABS(ab.SCHEME_FEE_EUR)) )AS SF ,

CASE WHEN ab.CARD_TYPE_GROUP LIKE '%Debit%' THEN 'Debit'
WHEN ab.CARD_TYPE_GROUP LIKE '%Credit%' THEN 'Credit'
WHEN ab.CARD_TYPE_GROUP LIKE '%Prepaid%' THEN 'Prepaid'
END AS "CREDIT_DEBIT",

CASE WHEN ab.CARD_TYPE_GROUP LIKE '%Consumer%' THEN 'Consumer'
WHEN ab.CARD_TYPE_GROUP LIKE '%Commercial%' THEN 'Commercial'
END AS "CARD_TYPE"

FROM DATA.ACQUIRER_AIB_NEWGEN_FILES ab
WHERE ab.TRANSACTION_DATE BETWEEN '11-01-2017' AND '12-31-2017'
AND ab.SCHEME_FEE_EUR!=0
AND (ab.TRANSACTION_TYPE ='Sale')
AND (ab.INTERCHANGE_REGION='Intra-region'OR ab.INTERCHANGE_REGION='Inter-region' OR ab.INTERCHANGE_REGION='Domestic')
GROUP BY  ab.CONTRACT_ID, ab.ISSUER_COUNTRY, ab.CARD_BRAND, ab.CLEARING_COUNTRY_NAME,ab.TRANSACTION_TYPE,CREDIT_DEBIT,CARD_TYPE_GROUP,TRANSACTION_FLOW_EUR
'''
,conn)

### Save the AIB Query to hard drive to Prevent Repeated Query Pulls

In [ ]:
ab.to_csv('d:\\djohnson\\Desktop\\ab_bb\\ab_avg_sf_by_flow.csv')

In [125]:
ab=None #clear the ab query from memory

## AIB Import

In [187]:
ab=pd.read_csv('d:\\djohnson\\Desktop\\ab_bb\\ab_avg_sf_by_flow.csv')

In [188]:
ab=ab.drop('Unnamed: 0',axis=1) #drop the unnamed index from the AIB import

## Create a Working Copy of the Bambora Data

In [189]:
bb1=bb.copy() #create working copy of the Bambora data

### Remove Bambora Whitespace

In [190]:
bb1['ISSUER_COUNTRY_NAME']=bb1['ISSUER_COUNTRY_NAME'].str.strip()
bb1['MERCHANT_COUNTRY_NAME']=bb1['MERCHANT_COUNTRY_NAME'].str.strip()

### Remove Excess Naming from SQL

In [191]:
bb1=bb1.drop(['B_TRANSACTION_TYPE','C_TYPE','VOLUME','TRANSACTION_AMOUNT_EUR','SF_FIXED_FEE_RESULT','SF_PERC_FEE_RESULT'],axis=1)

## Rename Columns to Allow for Merge

In [192]:
bb1=bb1.rename(columns={'ISSUER_COUNTRY_NAME':'ISSUER_COUNTRY','MERCHANT_COUNTRY_NAME':'CLEARING_COUNTRY_NAME','CONSUMER_CORPORATE':'CARD_TYPE','CARD_SCHEME':'CARD_BRAND'})

In [193]:
print bb1.head()

               BAMBORA_REF MONTH DAY  YEAR   PERIOD  CONTRACT_ID  \
0  74541097315024958871649    11  11  2017  2017-11     5,859.00   
1  74541097315024958872282    11  11  2017  2017-11     7,793.00   
2  74541097315024958871631    11  11  2017  2017-11     5,859.00   
3  25336247315024958871666    11  11  2017  2017-11     5,859.00   
4  74541097315024958872266    11  11  2017  2017-11     7,793.00   

  ISSUER_COUNTRY CLEARING_COUNTRY_NAME TRANSACTION_TYPE  CARD_BRAND  \
0    New Zealand                 China      TRANSACTION        Visa   
1    New Zealand                 China      TRANSACTION        Visa   
2    New Zealand                 China      TRANSACTION        Visa   
3    New Zealand                 China      TRANSACTION  Mastercard   
4    New Zealand                 China      TRANSACTION        Visa   

  CREDIT_DEBIT CARD_TYPE  FLOW   SF  SF_FIXED_FEE  SF_PERC_FEE  
0        Debit  Consumer 20.00 0.18          0.10         0.01  
1        Debit  Consumer 16.00 0.1

## Indentify Congruent Lanes

In [194]:
bb1_contracts=bb1[['CONTRACT_ID','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME','CARD_BRAND','CREDIT_DEBIT','CARD_TYPE']].drop_duplicates()

### Remove Excess Naming

In [195]:
ab=ab.drop(['TRANSACTION_TYPE','TRANSACTION_DATE'],axis=1)

In [196]:
print ab.head()

   CONTRACT_ID  ISSUER_COUNTRY  CARD_BRAND CLEARING_COUNTRY_NAME   FLOW   SF  \
0         5100         Ireland        Visa               Germany  44.00 0.02   
1         4235          Canada        Visa        United Kingdom   4.00 0.09   
2         1423  United Kingdom        Visa        United Kingdom 159.00 0.06   
3         4235         Finland        Visa        United Kingdom   6.00 0.02   
4         2443     Switzerland  MasterCard                 Italy  40.00 0.22   

  CREDIT_DEBIT CARD_TYPE  
0        Debit  Consumer  
1       Credit  Consumer  
2       Credit  Consumer  
3        Debit  Consumer  
4       Credit  Consumer  


## Attach AIB Lanes to Active Bambora Lanes 
This is a left join that will show all active Bambora lanes on only those AIB lanes where there is congruence.

In [197]:
ab_df=pd.merge(bb1_contracts,ab,on=['CONTRACT_ID','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME','CARD_BRAND','CREDIT_DEBIT','CARD_TYPE'],how='left')

In [198]:
bb1_contracts=None #clear the merge data
ab=None #clear the merge data

### Only show those lanes where there is activity for AIB and Bambora
This is done but filtering to only those results that have scheme fee information

In [199]:
ab_df=ab_df[(ab_df['SF']>0)].drop_duplicates()

### Only Show the Median aggregated by lane, contract ID, Credit/Debit and Card type, Flow

In [200]:
ab_df=ab_df.groupby(['CONTRACT_ID','ISSUER_COUNTRY','CARD_BRAND','CLEARING_COUNTRY_NAME','CREDIT_DEBIT','CARD_TYPE','FLOW'],as_index=False)['SF'].median()

In [201]:
print ab_df.head()

   CONTRACT_ID ISSUER_COUNTRY CARD_BRAND CLEARING_COUNTRY_NAME CREDIT_DEBIT  \
0     1,946.00        Albania       Visa            Luxembourg       Credit   
1     1,946.00        Albania       Visa            Luxembourg       Credit   
2     1,946.00        Albania       Visa            Luxembourg       Credit   
3     1,946.00        Albania       Visa            Luxembourg       Credit   
4     1,946.00        Albania       Visa            Luxembourg       Credit   

  CARD_TYPE  FLOW   SF  
0  Consumer  0.00 0.08  
1  Consumer  1.00 0.08  
2  Consumer  2.00 0.08  
3  Consumer  3.00 0.09  
4  Consumer  4.00 0.09  


In [202]:
ab_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22731 entries, 0 to 22730
Data columns (total 8 columns):
CONTRACT_ID              22731 non-null float64
ISSUER_COUNTRY           22731 non-null object
CARD_BRAND               22731 non-null object
CLEARING_COUNTRY_NAME    22731 non-null object
CREDIT_DEBIT             22731 non-null object
CARD_TYPE                22731 non-null object
FLOW                     22731 non-null float64
SF                       22731 non-null float64
dtypes: float64(3), object(5)
memory usage: 1.6+ MB


## Setup the Bambora Baseline Lanes, Flows, and SFs

In [203]:
base=ab_df.drop(['FLOW','SF'],axis=1).drop_duplicates()

In [204]:
bbbase=pd.merge(base,bb1,on=['CONTRACT_ID','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME','CARD_BRAND','CREDIT_DEBIT','CARD_TYPE'],how='left')

In [205]:
bbbase.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_BRAND,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,BAMBORA_REF,MONTH,DAY,YEAR,PERIOD,TRANSACTION_TYPE,FLOW,SF,SF_FIXED_FEE,SF_PERC_FEE
0,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983622411,11,13,2017,2017-11,TRANSACTION,5.00,0.09,0.06,0.01
1,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983612529,11,13,2017,2017-11,TRANSACTION,12.00,0.13,0.06,0.01
2,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983727038,11,13,2017,2017-11,TRANSACTION,8.00,0.11,0.06,0.01
3,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024984047204,11,13,2017,2017-11,TRANSACTION,1.00,0.07,0.07,0.01
4,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024984169347,11,13,2017,2017-11,TRANSACTION,4.00,0.08,0.07,0.01


In [206]:
bbbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118177 entries, 0 to 118176
Data columns (total 16 columns):
CONTRACT_ID              118177 non-null float64
ISSUER_COUNTRY           118177 non-null object
CARD_BRAND               118177 non-null object
CLEARING_COUNTRY_NAME    118177 non-null object
CREDIT_DEBIT             118177 non-null object
CARD_TYPE                118177 non-null object
BAMBORA_REF              118177 non-null object
MONTH                    118177 non-null object
DAY                      118177 non-null object
YEAR                     118177 non-null object
PERIOD                   118177 non-null object
TRANSACTION_TYPE         118177 non-null object
FLOW                     118177 non-null float64
SF                       118177 non-null float64
SF_FIXED_FEE             118177 non-null float64
SF_PERC_FEE              118177 non-null float64
dtypes: float64(5), object(11)
memory usage: 15.3+ MB


### Link AIB and Bambora

In [207]:
ab_bb_comp=pd.merge(bbbase,ab_df,on=['CONTRACT_ID','ISSUER_COUNTRY','CARD_BRAND','CLEARING_COUNTRY_NAME','CREDIT_DEBIT','CARD_TYPE','FLOW'],how='left',suffixes=('_BB','_AVG_AB'))

In [208]:
ab_bb_comp.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_BRAND,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,BAMBORA_REF,MONTH,DAY,YEAR,PERIOD,TRANSACTION_TYPE,FLOW,SF_BB,SF_FIXED_FEE,SF_PERC_FEE,SF_AVG_AB
0,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983622411,11,13,2017,2017-11,TRANSACTION,5.00,0.09,0.06,0.01,0.10
1,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983612529,11,13,2017,2017-11,TRANSACTION,12.00,0.13,0.06,0.01,0.14
2,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983727038,11,13,2017,2017-11,TRANSACTION,8.00,0.11,0.06,0.01,0.11
3,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024984047204,11,13,2017,2017-11,TRANSACTION,1.00,0.07,0.07,0.01,0.08
4,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024984169347,11,13,2017,2017-11,TRANSACTION,4.00,0.08,0.07,0.01,0.09


## Insert the Signal logic

In [209]:
ab_bb_comp['SF_DIFF_%']=(ab_bb_comp['SF_BB']-ab_bb_comp['SF_AVG_AB'])/ab_bb_comp['SF_AVG_AB']

ab_bb_comp['SIGNAL']=''
ab_bb_comp['SIGNAL'][(ab_bb_comp['SF_DIFF_%']>.05)]='Signal'

In [210]:
ab_bb_comp.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_BRAND,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,BAMBORA_REF,MONTH,DAY,YEAR,PERIOD,TRANSACTION_TYPE,FLOW,SF_BB,SF_FIXED_FEE,SF_PERC_FEE,SF_AVG_AB,SF_DIFF_%,SIGNAL
0,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983622411,11,13,2017,2017-11,TRANSACTION,5.00,0.09,0.06,0.01,0.10,-0.07,
1,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983612529,11,13,2017,2017-11,TRANSACTION,12.00,0.13,0.06,0.01,0.14,-0.04,
2,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024983727038,11,13,2017,2017-11,TRANSACTION,8.00,0.11,0.06,0.01,0.11,-0.06,
3,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024984047204,11,13,2017,2017-11,TRANSACTION,1.00,0.07,0.07,0.01,0.08,-0.17,
4,"1,946.00",Albania,Visa,Luxembourg,Credit,Consumer,74541097317024984169347,11,13,2017,2017-11,TRANSACTION,4.00,0.08,0.07,0.01,0.09,-0.10,


### Final Output

In [211]:
ab_bb_comp=ab_bb_comp[['BAMBORA_REF','TRANSACTION_TYPE','CONTRACT_ID','MONTH','DAY','YEAR','PERIOD'
                       ,'CARD_BRAND','CARD_TYPE','CREDIT_DEBIT','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME'
                       ,'FLOW','SF_FIXED_FEE','SF_PERC_FEE','SF_BB','SF_AVG_AB','SF_DIFF_%','SIGNAL']]

In [212]:
ab_bb_comp.head()

,BAMBORA_REF,TRANSACTION_TYPE,CONTRACT_ID,MONTH,DAY,YEAR,PERIOD,CARD_BRAND,CARD_TYPE,CREDIT_DEBIT,ISSUER_COUNTRY,CLEARING_COUNTRY_NAME,FLOW,SF_FIXED_FEE,SF_PERC_FEE,SF_BB,SF_AVG_AB,SF_DIFF_%,SIGNAL
0,74541097317024983622411,TRANSACTION,"1,946.00",11,13,2017,2017-11,Visa,Consumer,Credit,Albania,Luxembourg,5.00,0.06,0.01,0.09,0.10,-0.07,
1,74541097317024983612529,TRANSACTION,"1,946.00",11,13,2017,2017-11,Visa,Consumer,Credit,Albania,Luxembourg,12.00,0.06,0.01,0.13,0.14,-0.04,
2,74541097317024983727038,TRANSACTION,"1,946.00",11,13,2017,2017-11,Visa,Consumer,Credit,Albania,Luxembourg,8.00,0.06,0.01,0.11,0.11,-0.06,
3,74541097317024984047204,TRANSACTION,"1,946.00",11,13,2017,2017-11,Visa,Consumer,Credit,Albania,Luxembourg,1.00,0.07,0.01,0.07,0.08,-0.17,
4,74541097317024984169347,TRANSACTION,"1,946.00",11,13,2017,2017-11,Visa,Consumer,Credit,Albania,Luxembourg,4.00,0.07,0.01,0.08,0.09,-0.10,


In [226]:
print 'total transactions'
print ab_bb_comp['BAMBORA_REF'].count()
print 'total signals'
print ab_bb_comp['BAMBORA_REF'][(ab_bb_comp['SIGNAL']=='Signal')].count()

total transactions
118177
total signals
2523


In [214]:
ab_bb_comp.to_csv('d:\\djohnson\\Desktop\\ab_bb_comp.csv')